In [1]:
# 因子分析模型(241)

from IPython.display import Latex
from IPython.display import display, Math, Latex


def print_latex(latex_str):
    display(Latex(latex_str))
    
    
print_latex(r'因子分析模型: $X=\mu+\Lambda F+\epsilon$')
print_latex(r'其中$E(F)=0,\quad E(\epsilon)=0,\quad Cov(F)=I_m,\quad D(\epsilon)=Cov(\epsilon)=diag(\sigma_1^2,\cdots,\sigma_m^2),\quad Cov(F, \epsilon)=0$')
print_latex(r'原始变量$X$的协方差矩阵分解: $Cov(X)=\Lambda\Lambda^T+diag(\sigma_1^2,\cdots,\sigma_m^2)$')
print_latex(r'载荷因子$\alpha_{ij}$反映第$i$个变量和第$j$个公共因子的相关系数. 绝对值越大相关的密切程度越高.')
print_latex(r'变量$X_i$的共同度记为$h_i^2=\sum\limits_{j=1}^m\alpha_{ij}^2$, 又有$1=h_i^2+\sigma_i^2$, 故$h_i^2$越接近1, 因子分析效果越好')
print_latex(r'$\Lambda$中各列平方和$S_j=\sum\limits_{i=1}^p\alpha_{ij}^2$, 用于衡量$F_j$的相对重要性.')
print_latex(r'因子载荷矩阵的估计方法: 1.主成分分析法(242页); ')
print_latex(r'通过因子旋转来直观的判断因子的实际意义')
print_latex(r'因子得分: 反过来把公共因子表示为原变量的线性组合.')
print_latex(r'因子得分函数: $F_j=c_j+\beta_{j1}X_1+\cdots+\beta_{jp}X_p,\ j=1,2,\cdots,m$')
print_latex(r'有1.巴特莱特因子得分(249页); 2.回归方法(250页)')

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

...

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>